In [1]:
import sys,os
import pickle
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
from IPython.display import Image
data_type = 'float32'
os.environ["THEANO_FLAGS"] = 'device=cuda,floatX=' + data_type + ',force_device=True' + ',dnn.enabled=False'
sys.path.insert(1, '/nfs/team205/vk7/sanger_projects/BayraktarLab/cell2location/')
import cell2location
#os.chdir("/nfs/team283/aa16/SpaceJam/")
os.chdir("/nfs/team205/vk7/sanger_projects/collaborations/SpaceJam")
import spacejam

Can not use cuDNN on context None: Disabled by dnn.enabled flag
Mapped name None to device cuda: Tesla V100-SXM2-32GB (0000:62:00.0)


Import our three models:

In [2]:
from spacejam.models import LocationModelWTAMultiExperimentHierarchicalGeneLevel

In [3]:
models = [spacejam.models.LocationModelWTAMultiExperimentHierarchicalGeneLevel.LocationModelWTAMultiExperimentHierarchicalGeneLevel]

Load and format data:

In [4]:
working_dir = '/lustre/scratch117/cellgen/team283/Kidney-Nanostring/'
adata_wta = sc.read_h5ad(f"{working_dir}Kidney_AnnData.h5ad")

/nfs/team283/vk7/software/miniconda3farm5/envs/pyro160/lib/python3.7/site-packages/anndata/_core/anndata.py:119: ImplicitModificationWarning: Transforming to str index.


In [5]:
reg_path = f"{working_dir}scrna/regression_model/RegressionGeneBackgroundCoverageTorch_46covariates_40268cells_10882genes/"
adata_snrna_raw = sc.read(f'{reg_path}sc.h5ad')

In [6]:
# Column name containing cell type annotations
covariate_col_names = 'celltype'

# Extract a pd.DataFrame with signatures from anndata object
inf_aver = adata_snrna_raw.raw.var.copy()
inf_aver = inf_aver.loc[:, [f'means_cov_effect_{covariate_col_names}_{i}' for i in adata_snrna_raw.obs[covariate_col_names].unique()]]
from re import sub
inf_aver.columns = [sub(f'means_cov_effect_{covariate_col_names}_{i}', '', i) for i in adata_snrna_raw.obs[covariate_col_names].unique()]
inf_aver = inf_aver.iloc[:, inf_aver.columns.argsort()]

# normalise by average experiment scaling factor (corrects for sequencing depth)
inf_aver = inf_aver * adata_snrna_raw.uns['regression_mod']['post_sample_means']['sample_scaling'].mean()

In [7]:
counts_negativeProbes = adata_wta.obsm['negProbes']
counts_nuclei = np.asarray(adata_wta.obs['AOINucleiCount']).reshape(len(adata_wta.obs['AOINucleiCount']),1)

In [8]:
adata_wta.raw = adata_wta

In [9]:
inf_aver.index = np.array(adata_snrna_raw.var.loc[inf_aver.index,'SYMBOL'])

Run cell2location:

In [10]:
for m in [models[0]]:
    cell2location.run_c2l.run_cell2location(inf_aver, adata_wta, 
            model_name=m,
            export_args={'path': f"{working_dir}results/",
                         'run_name_suffix': '_narrow_sensitivity'},
            train_args={'use_raw': True, 'n_iter': 20000, 'sample_name_col': 'SlideName'},
            model_kwargs={
            "Y_data" : counts_negativeProbes,
            "gene_level_prior": {'mean': 1 / 2, 'sd': 1 / 4, 'sample_alpha': 20},
            "cell_number_prior" : {'cells_per_spot': counts_nuclei, 
                                   'factors_per_spot': 7, 'combs_per_spot': 7},
            "cell_number_var_prior" : {'cells_mean_var_ratio': 1, 
                                       'factors_mean_var_ratio': 1, 'combs_mean_var_ratio': 1}})

### Summarising single cell clusters ###
### Creating model ### - time 0.0 min
### Analysis name: LocationModelWTAMultiExperimentHierarchicalGeneLevel_7experiments_33clusters_231locations_9551genes_narrow_sensitivity
### Training model ###


Finished [100%]: Average Loss = nan


Finished [100%]: Average Loss = nan


  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

/nfs/team205/vk7/sanger_projects/BayraktarLab/cell2location/cell2location/models/base/pymc3_model.py:449: MatplotlibDeprecationWarning: Passing non-integers as three-element position specification is deprecated since 3.3 and will be removed two minor releases later.
/nfs/team205/vk7/sanger_projects/BayraktarLab/cell2location/cell2location/models/base/pymc3_model.py:450: MatplotlibDeprecationWarning: Passing non-integers as three-element position specification is deprecated since 3.3 and will be removed two minor releases later.


### Sampling posterior ### - time 8.72 min


  0%|          | 0/24 [00:00<?, ?it/s]

### Saving results ###
### Ploting results ###
### Plotting posterior of W / cell locations ###
Some error in plotting with scanpy or `cell2location.plt.plot_factor_spatial()`
 KeyError('spatial')
### Done ### - time 9.57 min


In [11]:
for m in [models[0]]:
    cell2location.run_c2l.run_cell2location(inf_aver, adata_wta, 
            model_name=m,
            export_args={'path': f"{working_dir}results/",
                         'run_name_suffix': '_narrow_counts_nuclei'},
            train_args={'use_raw': True, 'n_iter': 20000, 'sample_name_col': 'SlideName'},
            model_kwargs={
            "Y_data" : counts_negativeProbes,
            "gene_level_prior": {'mean': 1 / 2, 'sd': 1 / 4, 'sample_alpha': 2},
            "cell_number_prior" : {'cells_per_spot': counts_nuclei, 
                                   'factors_per_spot': 7, 'combs_per_spot': 7},
            "cell_number_var_prior" : {'cells_mean_var_ratio': 10, 
                                       'factors_mean_var_ratio': 1, 'combs_mean_var_ratio': 1}})

### Summarising single cell clusters ###
### Creating model ### - time 0.0 min
### Analysis name: LocationModelWTAMultiExperimentHierarchicalGeneLevel_7experiments_33clusters_231locations_9551genes_narrow_counts_nuclei
### Training model ###


Finished [100%]: Average Loss = nan


Finished [100%]: Average Loss = nan


  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

/nfs/team205/vk7/sanger_projects/BayraktarLab/cell2location/cell2location/models/base/pymc3_model.py:449: MatplotlibDeprecationWarning: Passing non-integers as three-element position specification is deprecated since 3.3 and will be removed two minor releases later.
/nfs/team205/vk7/sanger_projects/BayraktarLab/cell2location/cell2location/models/base/pymc3_model.py:450: MatplotlibDeprecationWarning: Passing non-integers as three-element position specification is deprecated since 3.3 and will be removed two minor releases later.


### Sampling posterior ### - time 8.22 min


  0%|          | 0/24 [00:00<?, ?it/s]

### Saving results ###
### Ploting results ###
### Plotting posterior of W / cell locations ###
Some error in plotting with scanpy or `cell2location.plt.plot_factor_spatial()`
 KeyError('spatial')
### Done ### - time 9.02 min
